<a href="https://colab.research.google.com/github/mohamed-mkh15/Pose-estimation-Aruco-RGBD-camera-/blob/main/aruco_pose_estimation_RGBDcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting Aruco marker position and orientation

In [ ]:
## All imports
import pyrealsense2 as rs  # python lib to read data from the realsense camera
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rc('font',family='Times New Roman')
import matplotlib.image as mpimg
import glob
from IPython.display import clear_output

from scipy.spatial.transform import Rotation

## Camera calibratioCameran Method #1: saving images and using cv2 calibration methods

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
# 1- Save some color images for calibration
#-------------------------------------------------------------------------------------------------------------------

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

# Set the resolution for our D455 RGBD camera
resolution_width = 1280 # pixels
resolution_height = 720 # pixels
frame_rate = 30

# enable the stream
config.enable_stream(rs.stream.depth, resolution_width, resolution_height, rs.format.z16, frame_rate)
config.enable_stream(rs.stream.color, resolution_width, resolution_height, rs.format.bgr8, frame_rate)

# Start streaming
pipeline.start(config)

i=0
try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        align = rs.align(rs.stream.color)
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.first(rs.stream.color)

        if not color_frame:
            continue

        # Convert images to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())

        # save images in a folder
        cv2.imwrite('calib_images/calib.%05d.jpg' % (i),color_image)
        i+=1

finally:

    # Stop streaming
    pipeline.stop()

2- Import a chess board image and get it's corners using cv2

In [ ]:
# prepare object points
nx = 13 #13 # number of inside corners in x
ny = 8 #8 # number of inside corners in y

# Make a list of calibration images
images = glob.glob('calib_images/calib*.jpg')
# Select any index to grab an image from the list
idx = 1
# Read in the image
img = mpimg.imread(images[idx])

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
plt.imshow(img)

3- Apply interensic Calibration

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny * nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Step through the list and search for chessboard corners
for idx in range(len(images)):
    img = mpimg.imread(images[idx])
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, add object points, image points
    #if corners is not None:
    True_count = 0
    if ret:
#         objp = np.zeros((len(corners),3), np.float32)
#         print('--',idx , len(objp), len(corners))
        objpoints.append(objp)#objp
        imgpoints.append(corners)
        True_count+=1
print('True_count = ', True_count)

img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

Import a chess board image and get it's corners using cv2

In [ ]:
## Test one image
images = glob.glob('calib_images/calib*.jpg')
idx = 14
test_img = mpimg.imread(images[idx])
test_img_size = (test_img.shape[1], test_img.shape[0])

# Perform undistortion
undist = cv2.undistort(test_img, mtx, dist, None, mtx)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Camira calibration Method #2: using libersense own methods

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
# Read and view RealSense data and save internsic matrix
#-------------------------------------------------------------------------------------------------------------------

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

# Set the resolution for our D455 RGBD camera
resolution_width = 1280 # pixels
resolution_height = 720 # pixels
frame_rate = 30

# Set the chessboard parameters for calibration
chessboard_width = 9 # squares
chessboard_height = 14 # squares
square_size = 0.04 # meters
chessboard_params = [chessboard_height, chessboard_width, square_size]

# enable the stream
config.enable_stream(rs.stream.depth, resolution_width, resolution_height, rs.format.z16, frame_rate)
config.enable_stream(rs.stream.color, resolution_width, resolution_height, rs.format.bgr8, frame_rate)


# Start streaming
cfg = pipeline.start(config)

# calculate intrinsic matrix
profile = cfg.get_stream(rs.stream.color)  # Fetch stream profile for color stream
intr = profile.as_video_stream_profile().get_intrinsics()
print("intr: ", intr)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        align = rs.align(rs.stream.color)
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.first(rs.stream.color)
        depth_frame = aligned_frames.get_depth_frame() # aligned depth frame

        #depth_frame = frames.get_depth_frame()
        #color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)

finally:

    # Stop streaming
    pipeline.stop()

In [ ]:
## Test one image
images = glob.glob('calib_images/calib*.jpg')
idx = 14
test_img = mpimg.imread(images[idx])
test_img_size = (test_img.shape[1], test_img.shape[0])

# Perform undistortion
mtx = np.array([[intr.fx, 0.     , intr.ppx],
                [0.    , intr.fy, intr.ppy],
                [0.    , 0.     , 1.     ]])

dist = np.array(intr.coeffs)
undist = cv2.undistort(test_img, mtx, dist, None, mtx)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)



## Now apply pose estimations using aruco code

Define pose estimation function

In [ ]:
def pose_esitmation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients, marker_length = 0.049):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()


    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters,
        cameraMatrix=matrix_coefficients,
        distCoeff=distortion_coefficients)

    # If markers are detected
    found=False
    if len(corners) > 0:
        found=True
        for i in range(0, len(ids)):
            # Estimate pose of each marker and return the values rvec and tvec---(different from those
            #of camera coefficients)
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], marker_length , matrix_coefficients,
                                                                           distortion_coefficients)
            # Draw a square around the markers
            cv2.aruco.drawDetectedMarkers(frame, corners)

            # Draw Axis
            cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)

            RR = Rotation.from_rotvec(rvec[0])
            np.save("rvec", np.array(RR.as_matrix()))
            np.save("tvec", tvec.reshape(3,-1))
            RR= Rotation.from_rotvec(rvec[0])

    if found:
        return frame, np.array(RR.as_matrix()), tvec.reshape(3,-1), found
    else:
        return frame, np.zeros(3),np.zeros(3), found

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
# Validate pose of aruco marker
#-------------------------------------------------------------------------------------------------------------------
aruco_dict_type = cv2.aruco.DICT_5X5_100

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

# Set the resolution for our D455 RGBD camera
resolution_width = 1280 # pixels
resolution_height = 720 # pixels
frame_rate = 30

# enable the stream
config.enable_stream(rs.stream.depth, resolution_width, resolution_height, rs.format.z16, frame_rate)
config.enable_stream(rs.stream.color, resolution_width, resolution_height, rs.format.bgr8, frame_rate)


# Start streaming
cfg = pipeline.start(config)

# calculate intrinsic matrix
profile = cfg.get_stream(rs.stream.color)  # Fetch stream profile for color stream
intr = profile.as_video_stream_profile().get_intrinsics()
mtx = np.array([[intr.fx, 0.     , intr.ppx],
                [0.    , intr.fy, intr.ppy],
                [0.    , 0.     , 1.     ]])

dist = np.array(intr.coeffs)
print("intr: ", intr)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        align = rs.align(rs.stream.color)
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.first(rs.stream.color)
        depth_frame = aligned_frames.get_depth_frame() # aligned depth frame

        #depth_frame = frames.get_depth_frame()
        #color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))


        # calculate position
        output,rvec_marker,tvec_marker,found = pose_esitmation(color_image, aruco_dict_type, mtx, dist, 0.049)

        if found:
            cube = Rotation.from_matrix(rvec_marker)
            #thetay=np.arccos(rvec_markertool[0,1,2])*180/np.pi
            #thetax=np.arccos(rvec_marker[0,0,2])*180/np.pi

            T_CB = np.array([[rvec_base[0,0,0],rvec_base[0,0,1],rvec_base[0,0,2],tvec_base[0,0]],[rvec_base[0,1,0],rvec_base[0,1,1],rvec_base[0,1,2],tvec_base[1,0]],  [rvec_base[0,2,0],rvec_base[0,2,1],rvec_base[0,2,2],tvec_base[2,0]],[0,0,0,1]])
            T_CM = np.array([[rvec_marker[0,0,0],rvec_marker[0,0,1],rvec_marker[0,0,2],tvec_marker[0,0]],[rvec_marker[0,1,0],rvec_marker[0,1,1],rvec_marker[0,1,2],tvec_marker[1,0]],[rvec_marker[0,2,0],rvec_marker[0,2,1],rvec_marker[0,2,2],tvec_marker[2,0]],[0,0,0,1]])
            T_transl = np.array([[1,0,0,0.1553],[0,1,0,0],[0,0,1,-0.011],[0,0,0,1]])
    #         T_transl=np.array([[1,0,0,0.425],[0,1,0,-0.4],[0,0,1,-0.0105+0.0295],[0,0,0,1]])
            T_BC=np.linalg.inv(T_CB)
            T_final=T_transl @ T_BC @ T_CM
            aru_x = T_final[0,3]
            aru_y = T_final[1,3]
            aru_z = T_final[2,3]
            #print("x=",T_final[0,3],"y=",T_final[1,3],"z=",T_final[2,3])
            print("x=",aru_x, "y=",aru_y,"z=",aru_z)
        cv2.imshow('Estimated Pose', output)



        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)

finally:

    # Stop streaming
    pipeline.stop()